In [11]:
import requests as r
import pandas as pd
import ujson as json
import shelve
import wikipedia

In [3]:
#source = "https://en.wikipedia.org/api.php?action=query&prop=revisions&rvprop=content&format=jsonfm&titles=Main%20Page"

In [4]:
def all_edits():
    round = 0
    while True:
        user = "DemocraticLuntz"
        source = "https://en.wikipedia.org/w/api.php?action=query&"
        props = "ids|title|timestamp|comment|size|flags"
        limit = 100
        body = "list=usercontribs&ucuser={}&uclimit={}&ucdir=older&format=json&ucprop={}".format(user,limit,props)
        if round > 0:
            body += "&uccontinue=" + results["continue"]["uccontinue"]
        page = r.get(source + body)
        results = json.loads(page.content)
        try:
            for contrib in results['query']['usercontribs']:
                yield contrib
        except StopIteration:
            break
        round += 1


In [12]:
all_of_them = []
import random
for edit in all_edits():
    all_of_them.append(edit)
    if not "svg" in edit["title"]:
        if ", Maine" in edit["title"]:
            if random.random() < .01:
                print edit["title"]

South Portland, Maine
Hibberts Gore, Maine
West Central Franklin, Maine
North Franklin, Maine
Millinocket, Maine
Hartland, Maine
East Hancock, Maine


KeyError: 'continue'

In [13]:
len(all_of_them)

140628

In [14]:
places = [z for z in all_of_them if not "svg" in z["title"]]

In [15]:
p = wikipedia.page(places[20]["title"])

In [22]:
city_shelf = shelve.open("all_places.shelf")

In [71]:
maine_cities = list(set([c["title"] for c in all_of_them if (", " in c['title'] or ", Vermont" in c['title'] or ", New Hampshire" in c['title'])]))
print len(maine_cities)

28596


In [129]:
import regex as re

In [63]:
city_shelf.keys()[0]

"('Airmont', 'village', 'Rockland County', 'New York')"

In [67]:
city_shelf = shelve.open("new_places.shelf")

In [160]:
def parse_row(row):
    try:
        y = int(row[:4])
        pop = int(row[5:])
    except:
        return None
    return (y,pop)

def parse_bars(row):
    try:
        _,y,p = row.split("|")
        return((int(y),int(p)))
    except:
        return None

def town_pops(article_title):
    try:
        content = city_shelf[article_title.encode("utf-8")]
        if "#REDIRECT" in content and len(content) < 100:
            content = content.split("\n",1)[0]
            old = article_title
            article_title = re.sub(r".*REDIRECT \[\[(.*)\]\][.\n]*",r"\1",content)
            print u"redirecting {} to {}".format(old,article_title)
            _ = city_shelf["fake key just to trigger a new search foobah"]
    except KeyError:
        if random.random() < .05:
            print article_title
        p = r.get(u"https://en.wikipedia.org/w/index.php?action=raw&title={}".format(article_title))
        content = p.content
        city_shelf[article_title.encode("utf-8")] = content
    if "Census population\n|" in content:
        start = content.index("US Census population\n|") + 13
        method = parse_row
        splitter="|"
    elif "USCensusPop\n|" in content:
        start = content.index("USCensusPop\n|") + 11
        method = parse_row
        splitter="|"

    elif "Historical populations\n|type= USA" in content:
        start = content.index("Historical populations\n|type= USA") + 34
        splitter = "\n"
        method = parse_bars
    else:
        raise TypeError(content)
    area = content[(start):(start + 500)]
    bits = area.split(splitter)
    return [pair for pair in map(method,bits) if pair is not None]


In [72]:
random.shuffle(maine_cities)
all_pops = map(town_pops,maine_cities)

Macon County, Georgia
Warren Park, Indiana
Eunice, Louisiana
Bloomfield Township, Crawford County, Pennsylvania
Logansport, Indiana
Clark, New Jersey
Chesterfield Township, Macoupin County, Illinois
Elmore, Ohio
Benton Harbor, Michigan
Portia, Arkansas
Cattaraugus, New York
Hindman, Kentucky
Ira, New York
Corning, Kansas
Palm Beach Gardens, Florida
Frankfort, Kansas
Danville, Iowa
Washoe County, Nevada
Sublette County, Wyoming
Spiro, Oklahoma
Greene County, New York
Iroquois Township, Iroquois County, Illinois
Huron, Ohio
Sycamore, South Carolina
Belmond, Iowa
Karthaus Township, Clearfield County, Pennsylvania
Scobey, Montana
Pattison, Texas
Lincoln, Michigan
Bratenahl, Ohio
Cleveland County, Arkansas
Delaware County, Iowa
Anderson Township, Clark County, Illinois
Country Life Acres, Missouri
Ellsworth, Kansas
Edgewood, Florida
Galena, Missouri
Wallowa County, Oregon
Kincaid, Kansas
Hardin County, Iowa
Middletown, Orange County, New York
Elm Springs, Arkansas
Englishtown, New Jersey
He

In [27]:
print maine_cities[1033]
all_pops[1033]

Sugar Hill, New Hampshire


[(1970, 336), (1980, 397), (1990, 464), (2000, 563), (2010, 563)]

In [73]:
len(all_pops)

28596

In [90]:
n = random.randint(0,len(all_pops)-1)
print maine_cities[n]
print(all_pops[n])

Villa Grove, Illinois
[(1910, 1828), (1920, 2493), (1930, 2001), (1940, 2072), (1950, 2026), (1960, 2308), (1970, 2605), (1980, 2707), (1990, 2734), (2000, 2553), (2010, 2537)]


In [161]:
data = []

def coords(city):
    utf = city_shelf[city.encode("utf-8")]

    try:
        start = utf.index("{{coord|") + 8
    except:
        return utf
    coords = utf[start:start+25].split("|")
    try:
        if "." in coords[0] and "." in coords[1]:
            lat = float(coords[0])
            lon = float(coords[1])
        elif len(coords)>=4 and coords[1] in "NS" and coords[3] in "EW":
            lat = float(coords[0])
            lon = float(coords[2])                                   
        elif len(coords)==2:
            lat,lon = map(float,coords)
        elif coords[2]=="N":
            lat = float(coords[0]) + float(coords[1])/60
            lon = -1 * (float(coords[3]) + float(coords[4])/60)
        else:
            lat = float(coords[0]) + float(coords[1])/60 + float(coords[2])/60/60
            lon = -1 * (float(coords[4]) + float(coords[5])/60 + float(coords[6])/60/60)
    except:
        print "BAD COORDS: {} for {}".format(coords,city)
        return (None,None)
    return (lat,lon)

for i,city in enumerate(maine_cities):
    if city in ["Sanford (CDP), Maine", "The Andersons, Inc.","Edmonton"]:
        continue
    lookup = city_shelf[city.encode("utf-8")]
    if "Infobox television" in lookup:
        continue
    try:
        pops = town_pops(city)
    except TypeError:
        if random.random() < .01:
            print u"Error with {} at point {}".format(city,i)
        continue
    coordinates = coords(city)
    if len(coordinates) > 2:
        coordinates = (None,None)
    for pop in pops:
        data.append((city,coordinates[0],coordinates[1],pop[0],pop[1]))
    


redirecting Davistown, Kentucky to Fairview, Kentucky
redirecting Universal Forest Products, Inc. to Universal Forest Products
redirecting Batavia Town, Genesee County, New York to Batavia (town), New York
redirecting Dansville (town), Steuben County, New York to Dansville, Steuben County, New York
redirecting Keurig Green Mountain, Inc. to Keurig Green Mountain
redirecting Middletown Town, Delaware County, New York to Middletown, Delaware County, New York
redirecting Pinkhams Grant, New Hampshire to Pinkham's Grant, New Hampshire
redirecting Fulton (city), Oswego County, New York to Fulton, Oswego County, New York
redirecting Clinton Town, Clinton County, New York to Clinton, Clinton County, New York
redirecting Dickinson Town, Broome County, New York to Dickinson, Broome County, New York
redirecting Itron, Inc. to Itron
redirecting Hopewell city, Virginia to Hopewell, Virginia
redirecting Northampton Town, Fulton County, New York to Northampton, Fulton County, New York
redirecting Ci

In [158]:
print city
town_pops(city)
coords(city)

Enfield, Connecticut
['region:US-CT', 'format=dms', 'd']


ValueError: could not convert string to float: region:US-CT

In [162]:
del city_shelf[city.encode("utf-8")]

In [166]:
frame = pd.DataFrame(data,columns=["city","lat","lon","year","pop"])
frame.to_csv("/home/bschmidt/9943478/Maine.csv",encoding="utf-8")

In [185]:
town_pops("Detroit")

[(1820, 1422),
 (1830, 2222),
 (1840, 9102),
 (1850, 21019),
 (1860, 45619),
 (1870, 79577),
 (1880, 116340),
 (1890, 205876),
 (1900, 285704),
 (1910, 465766),
 (1920, 993678),
 (1930, 1568662),
 (1940, 1623452),
 (1950, 1849568),
 (1960, 1670144),
 (1970, 1514063),
 (1980, 1203368),
 (1990, 1027974),
 (2000, 951270),
 (2010, 713777)]

In [188]:
zip(town_pops("Toledo, Ohio"),town_pops("Detroit"))

[((1840, 1222), (1820, 1422)),
 ((1850, 3829), (1830, 2222)),
 ((1860, 13768), (1840, 9102)),
 ((1870, 31584), (1850, 21019)),
 ((1880, 50137), (1860, 45619)),
 ((1890, 81434), (1870, 79577)),
 ((1900, 131822), (1880, 116340)),
 ((1910, 168497), (1890, 205876)),
 ((1920, 243164), (1900, 285704)),
 ((1930, 290718), (1910, 465766)),
 ((1940, 282349), (1920, 993678)),
 ((1950, 303616), (1930, 1568662)),
 ((1960, 318003), (1940, 1623452)),
 ((1970, 383818), (1950, 1849568)),
 ((1980, 354635), (1960, 1670144)),
 ((1990, 332943), (1970, 1514063)),
 ((2000, 313619), (1980, 1203368)),
 ((2010, 287208), (1990, 1027974))]


map(parse_bars,rows)

In [198]:
easy_lookup = {}

In [199]:
for (k,v) in zip(maine_cities,all_pops):
    try:
        z = dict(v)
    except ValueError:
        continue
    lookup = []
    for y in range(1790,2010,10):
        try:
            lookup.append(z[y])
        except KeyError:
            lookup.append(0)
    easy_lookup[tuple(lookup)] = k

In [249]:
TN = pd.read_csv("wiki_census/TNPlaces.csv")

all_rows = []

for row in TN.iterrows():
    z = row[1].to_dict()
    lookup = []
    for y in range(1790,2010,10):
        try:
            lookup.append(z[str(y)])
        except KeyError:
            lookup.append(0)
    try:
        z["match"] = easy_lookup[tuple(lookup)]
    except KeyError:
        z["match"] = "NOOOOOTHING"
    all_rows.append(z)
    

In [230]:
easy_lookup[tuple(lookup)]

u'Adams, Tennessee'

In [250]:
pd.DataFrame(all_rows)[pd.DataFrame(all_rows).match =="NOOOOOTHING"]

,1790,1800,1810,1820,1830,1840,1850,1860,1870,1880,...,1970,1980,1990,2000,2010,2014,County,Place,State,match
67,0,0,0,0,0,0,0,0,0,0,...,563,418,362,511,354,333,Polk County,Copperhill,Tennessee,NOOOOOTHING
220,0,0,0,0,5566,6929,10165,16988,25865,43350,...,426029,455651,488374,545524,601222,644014,Davidson County,Nashville-Davidson metropolitan government (ba...,Tennessee,NOOOOOTHING
228,0,0,0,0,0,0,0,0,0,145,...,0,0,0,3099,5861,6423,Williamson County,Nolensville,Tennessee,NOOOOOTHING
331,0,0,0,0,0,0,0,0,0,236,...,1983,2109,1824,2228,2449,2420,Wayne County,Waynesboro,Tennessee,NOOOOOTHING


In [242]:
print city_shelf["Waynesboro, Tennessee"]

{{Infobox settlement
|official_name            = Waynesboro, Tennessee
|settlement_type          = [[City]]
|nickname                 =
|motto                    =''Progressing toward our future, through an understanding of our past''<ref name=official>[http://www.cityofwaynesboro.org/ Waynesboro official website].  Retrieved: 1 March 2013.</ref>

<!-- Images -->
|image_skyline            = Wayne-County-Courthouse-SE-tn2.jpg
|imagesize                = 250px
|image_caption            = Wayne County Courthouse
|image_flag               =
|image_seal               =

<!-- Maps -->
|image_map                = TNMap-doton-Waynesboro.PNG
|mapsize                  = 250px
|map_caption              = Location of Waynesboro, Tennessee
|image_map1               =
|mapsize1                 =
|map_caption1             =

<!-- Location -->
|subdivision_type         = [[List of countries|Country]]
|subdivision_name         = [[United States]]
|subdivision_type1        = [[Political divisions of the

In [246]:
pd.DataFrame(all_rows).irow(331)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: irow(i) is deprecated. Please use .iloc[i]
  if __name__ == '__main__':


1790                 0
1800                 0
1810                 0
1820                 0
1830                 0
1840                 0
1850                 0
1860                 0
1870                 0
1880               236
1890               239
1900                 0
1910                 0
1920                 0
1930                 0
1940               768
1950              1147
1960              1343
1970              1983
1980              2109
1990              1824
2000              2228
2010              2449
2014              2420
County    Wayne County
Place       Waynesboro
State        Tennessee
match             None
Name: 331, dtype: object